# Setup/Imports

In [1]:
import pickle
import keras
import tensorflow as tf
from keras import backend as K
import numpy as np
import sys
import os
sys.path.append(os.path.abspath('../'))
from helpers.data_generator import process_data, DataGenerator
from helpers.custom_losses import denorm_loss, hinge_mse_loss
from helpers.custom_losses import percent_correct_sign, baseline_MAE
from models.LSTMConv2D import get_model_lstm_conv2d, get_model_simple_lstm
from models.LSTMConv2D import get_model_linear_systems, get_model_conv2d
from utils.callbacks import CyclicLR, TensorBoardWrapper
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from time import strftime, localtime
import matplotlib
from matplotlib import pyplot as plt

from helpers.normalization import normalize, denormalize, renormalize
import tkinter as tk
from tkinter import filedialog
root = tk.Tk()
root.withdraw()

Using TensorFlow backend.


''

In [2]:
num_cores = 4
config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,
                        inter_op_parallelism_threads=num_cores, 
                        allow_soft_placement=True,
                        device_count = {'CPU' : 1,
                                        'GPU' : 0})
session = tf.Session(config=config)
K.set_session(session)

In [3]:
%matplotlib inline
font={'family': 'Times New Roman',
      'size': 10}
plt.rc('font', **font)
matplotlib.rcParams['figure.facecolor'] = (1,1,1,1)

# Functions

In [4]:
def hinge_loss(delta_true, delta_pred):
    """Finds the percentage of the time the prediction has the right sign
    """
    return np.sum(np.maximum(np.sign(delta_true*delta_pred),0))/delta_true.size

def batch_hinge(model,generator,param_dict,sig):
    """Finds the percentage of the time the prediction has the right sign
    """    
    nbatches = len(generator)
    err = 0
    for ind in range(nbatches):
        inputs, targets = generator[ind]
        predictions = model.predict_on_batch(inputs)
        full_pred, full_true, delta_pred, delta_true, baseline = batch_denorm(inputs,targets,predictions,param_dict,sig)
        err += hinge_loss(delta_true,delta_pred)
    return err/nbatches    

def batch_denorm(inputs,targets,predictions,param_dict,sig):
    """Denormalizes and calculates deltas, prep for plotting and analysis
    """
    targets = targets['target_' + sig]
    baseline = inputs['input_' + sig][:,-1]
    predictions = predictions[sig]
    if predict_deltas:
        full_pred = predictions + baseline
        full_true = targets + baseline
    else:
        full_pred = predictions
        full_true = targets
    denorm_baseline = denormalize(baseline, param_dict[sig])
    denorm_full_pred = denormalize(full_pred, param_dict[sig])
    denorm_full_true = denormalize(full_true, param_dict[sig])
    denorm_delta_pred = denorm_full_pred - denorm_baseline
    denorm_delta_true = denorm_full_true - denorm_baseline
    return denorm_full_pred, denorm_full_true, denorm_delta_pred, denorm_delta_true, denorm_baseline

def plot_batch(y_true, y_pred, baseline, psi, labels, axlabels,shots,times):
    batch_size = y_true.shape[0]
    ncols = 4
    nrows = int(batch_size/ncols)
    figsize = (20,10*batch_size/10)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    ax = ax.flatten()
    # Plot predictions and true deltas
    for i in range(batch_size):
        ax[i].plot(psi,y_true[i], label=labels[0])
        ax[i].plot(psi,y_pred[i], label=labels[1])
        ax[i].plot(psi,baseline[i], label=labels[2])
        ax[i].title.set_text('Shot ' + str(int(shots[i])) + ', time ' + str(int(times[i])))
        ax[i].set_xlabel(axlabels[0])
        ax[i].set_ylabel(axlabels[1])
        ax[i].set_ylim(-2*normalization_params[sig]['std'],2*normalization_params[sig]['std'])
        ax[i].legend(loc=0)
    plt.tight_layout()
    return fig,ax

# Analysis

In [7]:
file_path = filedialog.askopenfilename(title='Saved Model File',filetypes = [("hdf5 files","*.h5"),("all files","*.*")])
model = keras.models.load_model(file_path, compile=False)
print('loaded model: ' + file_path.split('/')[-1])
file_path = file_path[:-3] + 'params.pkl'
with open(file_path, 'rb') as f:
     analysis_params = pickle.load(f, encoding='latin1')
print('loaded dict: ' + file_path.split('/')[-1])
print('with parameters: ' + str(analysis_params.keys()))
locals().update(analysis_params)

loaded model: model-conv2d_profiles-temp-dens-rotation-press-itemp-ffprime_act-pinj-curr-tinj-gasA_targ-temp_norm-StandardScaler_profLB-1_actLB-8_activ-relu_nshots-1000_31Jul19-14-05.h5


FileNotFoundError: [Errno 2] No such file or directory: '/home/fouriest/SCHOOL/Princeton/PPPL/model-conv2d_profiles-temp-dens-rotation-press-itemp-ffprime_act-pinj-curr-tinj-gasA_targ-temp_norm-StandardScaler_profLB-1_actLB-8_activ-relu_nshots-1000_31Jul19-14-05params.pkl'

In [6]:
train_frac = 1
val_frac = 0
nshots = 1000
traindata, valdata, param_dict = process_data(rawdata, sig_names, normalization_method,
                                              window_length, window_overlap, max(
                                                  profile_lookback, actuator_lookback),
                                              lookahead, sample_step, uniform_normalization,
                                              train_frac, val_frac, nshots,verbose=1)
traindata = denormalize(traindata, param_dict)
traindata = renormalize(traindata, normalization_params)

Loading
Signals: curr, dens, ffprime, gasA, itemp, pinj, press, rotation, temp, tinj


AttributeError: 'int' object has no attribute 'values'

In [ ]:
batch_size = 12
train_generator = DataGenerator(traindata, batch_size, input_profile_names,
                                actuator_names, target_profile_names,
                                profile_lookback, actuator_lookback, lookahead,
                                predict_deltas,2)
steps_per_epoch = len(train_generator)

In [ ]:
batch_ind = np.random.randint(len(train_generator))
inputs, targets = train_generator[batch_ind]
shotnums = train_generator.cur_shotnum[:,max(profile_lookback, actuator_lookback)]
times = train_generator.cur_times[:,max(profile_lookback, actuator_lookback)]
pred = model.predict_on_batch(inputs)
# add deltas to future actuators
#predict on new actuators
predictions = {}
for i, sig in enumerate(target_profile_names):
    predictions[sig] = pred[i]
psi = np.linspace(0,1,profile_length)
full_pred = {}
full_true = {}
delta_pred = {}
delta_true = {}
baseline = {}
for sig in target_profile_names:
    full_pred[sig], full_true[sig], delta_pred[sig], delta_true[sig], baseline[sig] = batch_denorm(inputs,targets,predictions,
                                                                                                   normalization_params,sig)

In [ ]:
sig = 'temp'
fig, ax = plot_batch(delta_true[sig],delta_pred[sig],np.zeros_like(delta_true[sig]),
                     psi,['true','pred','baseline'],['psi','delta ' + sig], shotnums,times)  

In [ ]:
sig = 'dens'
fig, ax = plot_batch(delta_true[sig],delta_pred[sig],np.zeros_like(delta_true[sig]),
                     psi,['true','pred','baseline'],['psi','delta ' + sig], shotnums,times)  

In [ ]:
sig = 'press'
fig, ax = plot_batch(delta_true[sig],delta_pred[sig],np.zeros_like(delta_true[sig]),
                     psi,['true','pred','baseline'],['psi','delta ' + sig], shotnums,times)  

In [ ]:
sig = 'temp'
fig, ax = plot_batch(full_true[sig],full_pred[sig],baseline[sig],
                     psi,['true','pred','baseline'],['psi',sig], shotnums,times)  

In [ ]:
sig = 'dens'
fig, ax = plot_batch(full_true[sig],full_pred[sig],baseline[sig],
                     psi,['true','pred','baseline'],['psi',sig], shotnums,times)  

In [ ]:
sig = 'press'
fig, ax = plot_batch(full_true[sig],full_pred[sig],baseline[sig],
                     psi,['true','pred','baseline'],['psi',sig], shotnums,times)  

In [ ]:
with open(rawdata,'rb') as f:
    data = pickle.load(f, encoding='latin1')

In [ ]:
sig = 'rotation'
zrat = []
for shot in data.values():
    if sig in shot.keys():
        if shot[sig].size>0:
            zrat.append(np.count_nonzero(shot[sig])/shot[sig].size)

In [ ]:
data[164334].keys()